# The problem

TensorFlow Python ops give cryptic error messages. Often the exceptions arise from several stack levels down the TensorFlow codebase. Because of this, it is frequently not clear to the user what input constraints are violated and what should be done to correct the error.

This is particularly challenging for ops that are highly polymorphic in the combinations of shapes and dtypes they accept. Documentation often does not fully describe the legal inputs to ops. Finding out whether a particular call is legal must be done by trial and error in many cases.


# Some Examples





In [ ]:
!pip install ipywidgets
!pip install -e ./
!pip install pyctb
#!pip install --upgrade opschema

In [1]:
import opschema
opschema.explain('tf.gather_nd')

Schema for tf.gather_nd

Indexes

Index  Description           
b      batch                 
r      read location         
w      write location        
e      slice element         
c      read address component

Signatures

params  indices  return[0]
bre     bwc      bwe      

Index ranks

rank(b) Unconstrained                          
rank(r) in [1, 7]                              
rank(w) Unconstrained                          
rank(e) Unconstrained                          
rank(c) = 1                                    
rank(b,c,w) in [0, 10]     sum-range constraint
rank(b,e,r) in [0, 10]     sum-range constraint

Computed dimensions



DType Rules

indices.dtype in (int32, int64)
params.dtype in (int32, int64, float16, float32, float64)


In [2]:
import sys
from IPython.display import display
import ipywidgets as widgets
import opschema
import pyctb

# pyctb.on()

schema_list = opschema.list_schemas()
explain_out = widgets.Output()

dropdown = widgets.Dropdown(
    options=schema_list,
    value=schema_list[0],
    description='Ops:',
    disabled=False,
)

def explain_op(change):
  explain_out.clear_output()
  with explain_out:
    print(change.new)
    opschema.explain(change.new)

dropdown.observe(explain_op, names='value')

display('Choose one of the available Op Schemas to see its definition')
display(dropdown)
display(explain_out)

'Choose one of the available Op Schemas to see its definition'

Dropdown(description='Ops:', options=('tf.gather_nd', 'tf.nn.atrous_conv2d', 'tf.nn.atrous_conv2d_transpose', …

Output()

In [ ]:
# Instantiate a Schema (but do not wrap the TensorFlow op
wrapped_out = widgets.Output()

def run_wrapped(change):
  opschema.register(change.new)
  op = opschema.get(change.new)
  gen = op.generate_args()
  for _ in range(10):
    next(gen)
  input = next(gen)
  input = { k: v.value() for k, v in input.items() }
  wrapped_out.clear_output()
  with wrapped_out:
    try:
      op.wrapped_op(**input)
    except:
      print('\n')
      sys.excepthook(*sys.exc_info())

dropdown.observe(run_wrapped, names='value')

display(dropdown)
display(wrapped_out)



# The solution

OpCheck provides an API for defining the legal inputs to TensorFlow ops.  Using the API, one can create a 'schema' for a given op.  Once created, the schema can be used to generate legal and illegal inputs for the op, produce descriptions of the various constraints, and check the validity of a given set of inputs and issue an informative error message.



The schema definition itself must be tested against the TensorFlow op to confirm that it accurately reflects TensorFlow's own constraints.  To do this, the 





## The Inputs Generator

Once a TensorFlow op is registered, it is represented as an instance of `opcheck.api`, which provides the function `generate_args()`.  This is a generator that generates sets of input arguments for the op.  It will generate a complete set of valid inputs, as well as a defined set of invalid inputs.

## An API for concisely defining the schema for the op


